In [1]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from itertools import *
from gurobipy import *

import gurobipy as gp
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
%matplotlib inline
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()
import math
import datetime
import os

In [2]:
#Funciones traduccion de resultados MILP
def traducir(x):
    #Traducir informacion
    resultados = pd.DataFrame([var.varName for var in x.values() if var.X > 0.99])
    nombres = ['Maquina', 'Bloque1', 'Aplicacion1', 'Bloque2', 'Aplicacion2', 'Dia']
    resultados = resultados[0].str.split(',', expand = True).drop(columns = 0)
    resultados.columns = nombres
    resultados['Dia'] = resultados['Dia'].apply(lambda x: int(x))
    resultados['Bloque1'] = resultados['Bloque1'].apply(lambda x: x[2:-1])
    resultados['Bloque2'] = resultados['Bloque2'].apply(lambda x: x[2:-1])
    resultados['Aplicacion1'] = resultados['Aplicacion1'].apply(lambda x: x[2:-2])
    resultados['Aplicacion2'] = resultados['Aplicacion2'].apply(lambda x: x[2:-2])
    return(resultados)

def ciclos(resultados, J):
    #Revisar ciclos
    conexion = pd.DataFrame({'Connected' : repeat(0,len(J))}, index = [[j[0] for j in J],[j[1] for j in J]])
    conexion.loc[('Bloque 0', '-')] = 1
    res0 = resultados[resultados['Bloque1'] == 'Bloque 0']
    for i in res0.index:
        r = res0.loc[i]
        b = r['Bloque2']
        h = r['Aplicacion2']
        conexion.loc[(b,h)] = conexion.loc[(r['Bloque1'],r['Aplicacion1'])]
        while b != 'Bloque 0':
            res = resultados[(resultados['Bloque1'] == b) & (resultados['Aplicacion1'] == h)].iloc[0]
            conexion.loc[(res['Bloque2'],res['Aplicacion2'])] = conexion.loc[(b,h)]
            b = res['Bloque2']
            h = res['Aplicacion2']
    return(conexion)

#Modelo MILP
def MILP(rutaIns, rutaSol, instancia):
    startTime = time.time()
    
    #Lectura de parametros
    tProcesamiento = pd.read_csv(rutaIns + '/tProcesamiento.csv')
    fVelocidad = pd.read_csv(rutaIns + '/fVelocidad.csv')
    tDistancia = pd.read_csv(rutaIns + '/tDistancias.csv')
    inalcanzable = pd.read_csv(rutaIns + '/inalcanzable.csv')
    parametros = pd.read_csv(rutaIns + '/parametros.csv', index_col = 'parametro')
    
    with gp.Env() as env, gp.Model(env=env) as m:
        #Conjuntos
        I = [] #Conjunto de Máquinas
        B = [] #Conjunto de Bloques
        H = [] #Conjunto de Aplicaciones
        J = [] #Conjunto de Trabajos
        J0 = [] #Conjunto de trabajos sin bodega
        U = {} #Bloques inalcanzables por máquina i

        #Parámetros
        p = {} #Tiempos de procesamiento del trabajo j en la máquina más lenta
        v = {} #Factor de velocidad de la máquina i
        d = {} #Dia esperado de terminación del trabajo j
        a = {} #Dia esperado de inicio del trabajo j
        s = {} #Distantancia entre un par de bloques b1 y b2
        u = {} #Bloque b inalcanzable por máquina i
        T = range(1, int(parametros['valor']['numDias'] + 1))

        ##Creacion de Parámetros-------------------------

        #Info trabajos
        for k in tProcesamiento.index:
            b = tProcesamiento['Bloque'][k]
            h = tProcesamiento['Aplicacion'][k]
            j = (b,h)
            B.append(b)
            H.append(h)
            J.append(j)
            p[j] = tProcesamiento['tProcesamiento'][k]
            d[j] = tProcesamiento['dFin'][k]
            a[j] = tProcesamiento['dInicio'][k]
        B = list(np.unique(B))
        H = list(np.unique(H))
        H0 = H.copy()
        H0.remove('-')
        J0 = J.copy()
        J0.remove(('Bloque 0','-'))

        #Info máquinas
        for k in fVelocidad.index:
            i = fVelocidad['Maquina'][k]
            I.append(i)
            v[i] = fVelocidad['fVelocidad'][k]
            U[i] = []
        I = list(np.unique(I))

        #Info distancias
        for k in tDistancia.index:
            b1 = tDistancia['Bloque1'][k]
            b2 = tDistancia['Bloque2'][k]
            s[b1,b2] = tDistancia['Distancia'][k]

        #Info inalcanzables
        for k in inalcanzable.index:
            i = inalcanzable['Maquina'][k]
            b = inalcanzable['Bloque'][k]
            U[i].append(b)
            u[i,b] = inalcanzable['u'][k]
            
        #Variables
        x = m.addVars(product(I,J,J,T), vtype = GRB.BINARY, name = ['x,{0},{1},{2},{3}'.format(i,j,k,t) for i,j,k,t in product(I,J,J,T)])
        y = m.addVars(J0,vtype = GRB.BINARY)
        z = m.addVars(J0,vtype = GRB.BINARY)
        Dia = m.addVars(J0,vtype = GRB.CONTINUOUS)
        Earl = m.addVars(J0,vtype = GRB.CONTINUOUS)
        Tard = m.addVars(J0,vtype = GRB.CONTINUOUS)

        #Restricciones-----
        #Disponibilidad
        for i in I:
            m.addConstr(quicksum(x[i,j,k,t] + x[i,k,j,t] for t,j,k in product(T,[j for j in J if j[0] in U[i]],J)) == 0,
                        'Disponibilidad')
        #Asignación
        for j in J0:
            m.addConstr(quicksum(x[i,j,k,t] for t,i,k in product(T,I,J)) == 1, 'Asignacion')
        #Reciclo
        for t,i,j in product(T,I,J):
            m.addConstr(x[i,j,j,t] == 0, 'Reciclo')
        #Reciclo pares
        for t,i,j,k in product(T,I,J0,J0):
            m.addConstr(x[i,j,k,t] + x[i,k,j,t] <= 1, 'Reciclo')
        #Inicio-Fin
        for t,i in product(T,I):
            m.addConstr(quicksum(x[i,('Bloque 0','-'),j,t] for j in J) <= 1, 'Inicio-Fin1')
            m.addConstr(quicksum(x[i,('Bloque 0','-'),j,t] for j in J) == quicksum(x[i,j,('Bloque 0','-'),t] for j in J),
                        'Inicio-Fin2')
            m.addConstr(quicksum(x[i,j,k,t] for j,k in product(J0,J0)) <= len(J) * quicksum(x[i,('Bloque 0','-'),j,t] for j in J),
                        'Inicio-Fin3')
        #Ruteo
        for t,i,j in product(T,I,J):
            m.addConstr(quicksum(x[i,j,k1,t] for k1 in J) - quicksum(x[i,k2,j,t] for k2 in J) == 0, 'Ruteo')
        #Jornada Laboral
        for t,i in product(T,I):
            m.addConstr(quicksum(x[i,(b1,h1),(b2,h2),t] * (s[b1,b2] + p[(b1,h1)]/v[i]) for (b1,h1),(b2,h2) in product(J,J)) <= parametros['valor']['Q'],
                        'Jornada Laboral')
        #Definición Día
        for j in J0:
            m.addConstr(Dia[j] == quicksum(t * x[i,j,k,t] for t,i,k in product(T,I,J)), 'Dia')
        #Definición Earliness
        for j in J0:
            m.addConstr(a[j] - Dia[j] <= len(T) * y[j], 'Earl1')
            m.addConstr(Dia[j] - a[j] <= len(T) * (1 - y[j]), 'Earl2')
            m.addConstr(Earl[j] >= a[j] - Dia[j], 'Earl3')
            m.addConstr(Earl[j] <= a[j] - Dia[j] + len(T) * (1 - y[j]), 'Earl4')
            m.addConstr(Earl[j] <= len(T) * y[j], 'Earl5')
        #Definición Tardiness
        for j in J0:
            m.addConstr(Dia[j] - d[j] <= len(T) * z[j], 'Tard1')
            m.addConstr(d[j] - Dia[j] <= len(T) * (1 - z[j]), 'Tard2')
            m.addConstr(Tard[j] >= Dia[j] - d[j], 'Tard3')
            m.addConstr(Tard[j] <= Dia[j] - d[j] + len(T) * (1 - z[j]), 'Tard4')
            m.addConstr(Tard[j] <= len(T) * z[j], 'Tard5')

        #Función Objetivo
        m.setObjective(parametros['valor']['lambda1'] * quicksum(Earl[j] + Tard[j] for j in J0) +
                  parametros['valor']['lambda2'] * quicksum(x[i,(b1,h1),(b2,h2),t] * s[b1,b2]/parametros['valor']['Q']
                                                            for t,i,(b1,h1),(b2,h2) in product(T,I,J,J)), GRB.MINIMIZE)

        #Correr
        m.setParam('OutputFlag', 0)
        m.optimize()

        #Corte de subciclos
        conexion = ciclos(traducir(x), J)
        subC = list(conexion[conexion['Connected'] == 0].index)
        #Agregar cortes
        while len(subC) > 0:
            m.addConstr(quicksum(x[i,j,k,t] for t,i,j,k in product(T,I,subC,subC)) <= len(subC)-1, 'SubCiclos')
            m.optimize()
            conexion = ciclos(traducir(x), J)
            subC = list(conexion[conexion['Connected'] == 0].index)
        resultados = traducir(x)
    resultados = resultados.merge(tDistancia, how = 'left', on = ['Bloque1','Bloque2'])
    resultados['Distancia'] = resultados['Distancia'].apply(lambda x: x/parametros['valor']['Q'])
    resultados = resultados.merge(tProcesamiento, how = 'left', left_on = ['Bloque1','Aplicacion1'],
                                  right_on = ['Bloque','Aplicacion'])
    resultados.drop(['Bloque','Aplicacion'], axis = 1, inplace = True)
    resultados = resultados.merge(fVelocidad, how = 'left', on = ['Maquina'])
    resultados['ProcEfectivo'] = resultados['tProcesamiento']/resultados['fVelocidad']
    resultados['Earl-Tard'] = np.maximum(resultados['Dia']-resultados['dFin'],0) + np.maximum(resultados['dInicio']-resultados['Dia'],0)
    resultados['Earl-Tard'] = np.where(resultados['Bloque1'] == 'Bloque 0', 0, resultados['Earl-Tard'])
    orden = ['Dia'] + list(resultados.columns[:5]) + list(resultados.columns[[6,7,10,11,8,9,12]])
    resultados = resultados[orden]
    resultados = resultados.sort_values(['Dia','Maquina'])
    FO = parametros['valor']['lambda1']*sum(resultados['Earl-Tard']) + parametros['valor']['lambda2']*sum(resultados['Distancia'])
        
    tiempo = round(time.time() - startTime)
    resultados.to_csv(rutaSol + '/resultados MILP.csv', index = False)
    
    return pd.DataFrame({'Instancia':[instancia], 'FO':[FO], 'Tiempo':[tiempo]})


#Correr modelo:
def run(tam, nInstancias):
    metricas = pd.DataFrame()
    rutaIns = 'Instancias/' + tam
    rutaSol = 'Resultados/' + tam
    
    if not os.path.isdir(rutaSol):
        os.mkdir(rutaSol)
        
    for inst in range(1, nInstancias+1):
        instancia = '/Instancia ' + str(inst)
        rutaIns = 'Instancias/' + tam + instancia
        rutaSol = 'Resultados/' + tam + instancia

        if not os.path.isdir(rutaSol):
            os.mkdir(rutaSol)
        
        res = MILP(rutaIns, rutaSol, instancia[1:])
        metricas = metricas.append(res, ignore_index = True)
            
    metricas.to_csv('Resultados/' + tam + '/metricas MILP.csv', index = False)
    return metricas

In [ ]:
run('Pequena', 10)

Set parameter Username
Academic license - for non-commercial use only - expires 2022-06-09
Set parameter Username
Academic license - for non-commercial use only - expires 2022-06-09


/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40252/3970370943.py:213: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



Set parameter Username
Academic license - for non-commercial use only - expires 2022-06-09


/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40252/3970370943.py:213: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



Set parameter Username
Academic license - for non-commercial use only - expires 2022-06-09


...

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40252/3970370943.py:213: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



Set parameter Username
Academic license - for non-commercial use only - expires 2022-06-09


/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40252/3970370943.py:213: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



Set parameter Username
Academic license - for non-commercial use only - expires 2022-06-09


/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40252/3970370943.py:213: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

